<a href="https://colab.research.google.com/github/farazzashraf/LGMVIP--DataScience/blob/main/Music_Recommendation_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Music Recommendation System

In [41]:
# Importing the libraries
import pandas as pd
import numpy as np
from lightfm import LightFM
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_matrix
from lightfm.evaluation import precision_at_k

In [42]:
# Step 1: Loading the datasets
train_songs = pd.read_csv("train songs.csv")
song_extra_info = pd.read_csv("song_extra_info.csv")
songs_info = pd.read_csv("songs.csv")

# Dropping a na values
train_songs = train_songs.dropna()
song_extra_info = song_extra_info.dropna()
songs_info = songs_info.dropna()

In [43]:
# Step 2 Merging the dataset
merged_songs = train_songs.merge(song_extra_info[['song_id', 'name']], on='song_id', how='left')
merged_songs = merged_songs.merge(songs_info[['song_id', 'artist_name', 'lyricist']], on='song_id', how='left')

merged_songs = merged_songs.dropna()

# Step 3: creating a mapping of user and song ids to their repective indices
label_encoder = LabelEncoder()
merged_songs['msno'] = label_encoder.fit_transform(merged_songs['msno'])
merged_songs['song_id'] = label_encoder.fit_transform(merged_songs['song_id'])

user_mapping = {user_id: user_idx for user_idx, user_id in enumerate(merged_songs['msno'].unique())}
song_mapping = {song_id: song_idx for song_idx, song_id in enumerate(merged_songs['song_id'].unique())}

<ipython-input-43-787b131f467d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_songs['msno'] = label_encoder.fit_transform(merged_songs['msno'])
<ipython-input-43-787b131f467d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_songs['song_id'] = label_encoder.fit_transform(merged_songs['song_id'])


In [44]:
# Step 4: Creating interaction matrices
train_interactions = coo_matrix((np.ones(len(merged_songs), dtype=np.float32),
                                 (merged_songs['msno'], merged_songs['song_id'])),
                                shape=(len(user_mapping), len(song_mapping)))

# Step 5: Training the model
model = LightFM(loss='warp') # using the WARP loss function for implicit feedback data
model.fit(train_interactions, epochs=10, num_threads=2)

In [45]:
# Getting recommendation for the a user
user_id = int(input("User ID = "))
if user_id not in user_mapping:
  raise ValueError(f"User ID '{user_id}' is not present in the label encoder's mapping.")

user_index = user_mapping[user_id]
n_items = len(song_mapping)
scores = model.predict(user_index, np.arange(n_items))
top_item_indices = np.argsort(-scores)[:10]
top_item_ids = [k for k, v in song_mapping.items() if v in top_item_indices]
top_item_titles = merged_songs.loc[merged_songs['song_id'].isin(top_item_ids), ['name', 'artist_name', 'lyricist']]

# Getting the scores for the top recommendd songs
top_scores = scores[top_item_indices]

# Print the top recommend songs and scores vertically
print(f"Top recommendations for user {user_id}:")

for i, (song_title, artist_name, lyricist, score) in enumerate(zip(top_item_titles['name'], top_item_titles['artist_name'],
                                                    top_item_titles['lyricist'], top_scores), 1):
  print(f"{i}, Song: {song_title}")
  print(f"{i}, Artist: {artist_name}")
  print(f"{i}, Lyricist: {lyricist}")
  print(f"{i}, Score: {score:.2f}")
  print()

User ID = 45
Top recommendations for user 45:
1, Song: Me Too
1, Artist: Meghan Trainor
1, Lyricist: Meghan Trainor| Eric Frederic| Jacob Kasher Hindlin| Jason Desrouleaux| Peter Svensson
1, Score: 1.67

2, Song: Me Too
2, Artist: Meghan Trainor
2, Lyricist: Meghan Trainor| Eric Frederic| Jacob Kasher Hindlin| Jason Desrouleaux| Peter Svensson
2, Score: 1.46

3, Song: Me Too
3, Artist: Meghan Trainor
3, Lyricist: Meghan Trainor| Eric Frederic| Jacob Kasher Hindlin| Jason Desrouleaux| Peter Svensson
3, Score: 1.46

4, Song: Me Too
4, Artist: Meghan Trainor
4, Lyricist: Meghan Trainor| Eric Frederic| Jacob Kasher Hindlin| Jason Desrouleaux| Peter Svensson
4, Score: 1.43

5, Song: Me Too
5, Artist: Meghan Trainor
5, Lyricist: Meghan Trainor| Eric Frederic| Jacob Kasher Hindlin| Jason Desrouleaux| Peter Svensson
5, Score: 1.26

6, Song: Me Too
6, Artist: Meghan Trainor
6, Lyricist: Meghan Trainor| Eric Frederic| Jacob Kasher Hindlin| Jason Desrouleaux| Peter Svensson
6, Score: 1.25

7, Son